# Reconhecimento de entidades nomeadas

Uma grande empresa de advocacia está buscando soluções tecnológicas para otimizar o processamento de documentos jurídicos e melhorar a gestão dos processos em andamento.

Atualmente, o escritório lida com um volume significativo de contratos, petições, jurisprudências e outros documentos legais, mas não há uma forma eficiente de extrair automaticamente informações críticas desses textos, como nomes de partes envolvidas, datas relevantes e dispositivos legais citados.

Nosso papel nesse projeto é desenvolver um modelo de IA especializado em Reconhecimento de Entidades Nomeadas (NER), capaz de identificar e classificar automaticamente essas informações em textos jurídicos. Isso permitirá que os advogados e equipes jurídicas tenham acesso mais rápido a dados importantes, otimizando o tempo de análise e reduzindo o risco de erro humano, além de facilitar a gestão dos processos de forma mais eficiente.

## Leitura dos textos

Vamos utilizar dados de arquivos txt que estão compactados em um arquivo zip. Para abrir esses arquivos, vamos usar a biblioteca `zipfile`.

In [1]:
import zipfile

In [2]:
with zipfile.ZipFile('/content/texts.zip', 'r') as zip:
    print(zip.namelist())

['ADI2TJDFT.txt', 'adi3767.txt', 'Ag10000170733596001.txt', 'Ag10105170208398001.txt', 'AgAIRR11889820145030011.txt', 'AgCr10582160008758001.txt', 'AgRgSTJ1.txt', 'AgRgSTJ2.txt', 'AgRgTSE1.txt', 'AgRgTSE3.txt', 'AIAgRAgI6193ARAGUARIMG.txt', 'airr801422012.txt', 'AIRR3999520145020086.txt', 'AIRR15708820115050222.txt', 'AP00000794920137060006.txt', 'AP00001415620157010201.txt', 'AP00001441420167030203.txt', 'AP771420167080008PA.txt', 'CP32320177080008PA.txt', 'DespSEPLAGDF.txt', 'ED1STM.txt', 'ED1TJAC.txt', 'EDAgRgTSE2.txt', 'EDEDARR208420135040232.txt', 'EDRR1TST.txt', 'EEDRR9715120105020002.txt', 'ERR731004520105130003.txt', 'HC110260SP.txt', 'HC151914AgRES.txt', 'HC340624SP.txt', 'HC418951PR.txt', 'HC70000845920187000000.txt', 'lei11340.txt', 'Lei11788.txt', 'LoaDF2018.txt', 'Pet128TSE5.txt', 'Port77DF.txt', 'Rcl3495STJ.txt', 'REE5908TSE4.txt', 'REsp1583083RS.txt', 'RR474820145230056.txt', 'RR942006420095040028.txt', 'RR2574407120025020372.txt', 'TCU4687.txt', 'TSTRR16037920105200001.

In [3]:
with zipfile.ZipFile('/content/texts.zip', 'r') as zip:
    print(*zip.namelist(), sep = '\n')

ADI2TJDFT.txt
adi3767.txt
Ag10000170733596001.txt
Ag10105170208398001.txt
AgAIRR11889820145030011.txt
AgCr10582160008758001.txt
AgRgSTJ1.txt
AgRgSTJ2.txt
AgRgTSE1.txt
AgRgTSE3.txt
AIAgRAgI6193ARAGUARIMG.txt
airr801422012.txt
AIRR3999520145020086.txt
AIRR15708820115050222.txt
AP00000794920137060006.txt
AP00001415620157010201.txt
AP00001441420167030203.txt
AP771420167080008PA.txt
CP32320177080008PA.txt
DespSEPLAGDF.txt
ED1STM.txt
ED1TJAC.txt
EDAgRgTSE2.txt
EDEDARR208420135040232.txt
EDRR1TST.txt
EEDRR9715120105020002.txt
ERR731004520105130003.txt
HC110260SP.txt
HC151914AgRES.txt
HC340624SP.txt
HC418951PR.txt
HC70000845920187000000.txt
lei11340.txt
Lei11788.txt
LoaDF2018.txt
Pet128TSE5.txt
Port77DF.txt
Rcl3495STJ.txt
REE5908TSE4.txt
REsp1583083RS.txt
RR474820145230056.txt
RR942006420095040028.txt
RR2574407120025020372.txt
TCU4687.txt
TSTRR16037920105200001.txt
AC1TCU.txt
AC1TJAC.txt
AC1TJMG.txt
AC2.txt
ACORDAOTCU25052016.txt


In [4]:
with zipfile.ZipFile('/content/texts.zip', 'r') as zip:
    with zip.open('ADI2TJDFT.txt') as arquivo:
        texto = arquivo.read().decode('utf-8')

In [5]:
print(texto)

Órgão	:	Conselho Especial
Classe	:	ADI  Ação Direta de Inconstitucionalidade
N. Processo	:	2010002019357-4
Requerente(s)	:	PROCURADORA-GERAL DE JUSTIÇA DO DISTRITO FEDERAL E TERRITÓRIOS
Requerido(s)	:	PRESIDENTE DA CÂMARA LEGISLATIVA DO DISTRITO FEDERAL E OUTRO(S)
Relator 	:	Desembargador LÉCIO RESENDE
	EMENTA	
AÇÃO DIRETA DE INCONSTITUCIONALIDADE. LEIS DISTRITAIS N.º 747/1994 E 2018/1998. LEI COMPLEMENTAR DISTRITAL N.º 380/2001. INCONSTITUCIONALIDADE FORMAL. LEI ORGÂNICA DO DISTRITO FEDERAL. OCUPAÇÃO DE ÁREA PÚBLICA. COMPETÊNCIA PRIVATIVA DO GOVERNADOR DO DISTRITO FEDERAL. AÇÃO JULGADA PROCEDENTE EM RAZÃO DO VÍCIO FORMAL. Tanto o Decreto n.º 10.829/87, quanto a Portaria n.º 314/92, do Instituto Brasileiro do Patrimônio Cultural  IBPC, hoje Instituto do Patrimônio Histórico e Artístico Nacional  IPHAN, conferem ao Governador do Distrito Federal competência privativa para iniciar o processo legislativo, quando se tratar o tema de uso e ocupação do solo em todo o território do Distrito F

## Reconhecendo as entidades no texto

Para fazer o reconhecimento das entidades nos textos, vamos usar modelos da biblioteca `spacy`. Podemos utilizar modelos já treinados e personalizar esses modelos para atender ao nosso problema específico.

A documentação fornece detalhes de como fazer a instalação do modelo com base no sistema operacional e a língua que desejarmos.

https://spacy.io/usage

In [6]:
# !pip install -U spacy

In [7]:
# !python -m spacy download pt_core_news_sm

In [8]:
import spacy
import pt_core_news_sm

modelo_ner = pt_core_news_sm.load()

In [9]:
doc = modelo_ner(texto)

for entidade in doc.ents:
    print(f'{entidade.text} -> {entidade.label_}')

Órgão -> LOC
Conselho Especial
Classe -> ORG
DISTRITO FEDERAL E TERRITÓRIOS
Requerido(s -> LOC
DA CÂMARA -> ORG
DISTRITO FEDERAL E OUTRO(S)
Relator -> LOC
LEIS DISTRITAIS -> MISC
COMPLEMENTAR DISTRITAL -> MISC
FORMAL -> ORG
LEI ORGÂNICA -> MISC
DISTRITO FEDERAL -> LOC
ÁREA PÚBLICA -> LOC
DO GOVERNADOR -> MISC
DISTRITO FEDERAL -> LOC
VÍCIO FORMAL -> ORG
Decreto n.º 10.829/87 -> MISC
Portaria n.º 314/92 -> ORG
Instituto Brasileiro do Patrimônio Cultural  IBPC -> LOC
Instituto do Patrimônio Histórico e Artístico Nacional  IPHAN -> LOC
Governador do Distrito Federal -> LOC
Distrito Federal -> LOC
ACÓRDÃO -> ORG
Acordam os Desembargadores do Conselho Especial do Tribunal de Justiça do Distrito Federal -> MISC
Territórios LÉCIO RESENDE  Relator -> ORG
JOÃO MAIORISI -> PER
Vogal -> ORG
VIEIRA  Vogal -> LOC
SÉRGIO BITTENCOURT  Vogal -> ORG
LECIR MANOEL DA LUZ -> ORG
Vogal -> LOC
CARMELITA BRASIL -> ORG
Vogal -> LOC
WALDIR LEÔNCIO -> PER
J.J. COSTA CARVALHO -> PER
Vogal -> LOC
HUMBERTO -> LOC
V

In [10]:
import pandas as pd

In [11]:
entidades = []
labels = []

for entidade in doc.ents:
    entidades.append(entidade.text)
    labels.append(entidade.label_)

In [12]:
pd.DataFrame({'Entidade': entidades, 'Label': labels})

,Entidade,Label
0,Órgão,LOC
1,Conselho Especial\nClasse,ORG
2,DISTRITO FEDERAL E TERRITÓRIOS\nRequerido(s,LOC
3,DA CÂMARA,ORG
4,DISTRITO FEDERAL E OUTRO(S)\nRelator,LOC
...,...,...
851,PROCEDENTE,LOC
852,Leis Distritais,MISC
853,Lei Complementar,MISC
854,omnes,ORG


## Visualizando as entidades no texto

In [16]:
modelo_ner.get_pipe('ner').labels

('LOC', 'MISC', 'ORG', 'PER')

In [17]:
print('LOC', spacy.explain('LOC'))
print('MISC', spacy.explain('MISC'))
print('ORG', spacy.explain('ORG'))
print('PER', spacy.explain('PER'))

LOC Non-GPE locations, mountain ranges, bodies of water
MISC Miscellaneous entities, e.g. events, nationalities, products or works of art
ORG Companies, agencies, institutions, etc.
PER Named person or family.


In [18]:
spacy.displacy.render(doc, style = 'ent', jupyter=True)

In [19]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 59.2 MB/s eta 0:00:00
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.7.1
    Uninstalling en-core-web-sm-3.7.1:
      Successfully uninstalled en-core-web-sm-3.7.1
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [20]:
import en_core_web_sm

modelo_ner_ing = spacy.load('en_core_web_sm')
modelo_ner_ing.get_pipe('ner').labels

('CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART')